### Libraries

In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
import numpy as np

In [2]:
connection_args = {
    'host': '18.191.175.185', # You'll have to update this to your IP
    'user': 'ubuntu',    # username
    'dbname': 'mcnulty',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}
connection = pg.connect(**connection_args)
cursor = connection.cursor()

### Create Tables

Autopopulate the create table query with the proper column names and column types from the table schema file:

In [ ]:
with open('./Data/gw_tblschema.csv', mode = 'r') as filename:
    cols = filename.read()
    filename.close()
    
cols_list = [i.split(',')[1:3] for i in cols.split('\n')[1:-1]]
cols_text = ''
for i, txt in enumerate(cols_list):
    name = txt[0]
    col_type = txt[1]
    if col_type == 'STRING':
        col_type = 'TEXT'
    elif col_type == 'INTEGER':
        col_type = 'INT'
        
    if i == len(cols_list) - 1:
        cols_text += f'{name} {col_type}'
    else:
        cols_text += f'{name} {col_type}, '

Create baseball and gamepostwide tables in the SQL server:

In [ ]:
create_tblquery = f""" 
    CREATE TABLE baseball(
        row INT, 
        {cols_text}
    );"""

create_postwidetablequery = f""" 
    CREATE TABLE gamepostwide(
        row INT, 
        {cols_text}
    );"""

# cursor.execute('BEGIN;')
# cursor.execute(create_tblquery)
# cursor.execute(create_postwidetablequery)
# cursor.execute('commit;')

### EDA

In [ ]:
gw1_df['is_steal'] = gw1_df['description'].apply(lambda x: 0 if (type(x) != str or 'steal' not in x) else 1)
gw2_df['is_steal'] = gw2_df['description'].apply(lambda x: 0 if (type(x) != str or 'steal' not in x) else 1)
gw3_df['is_steal'] = gw3_df['description'].apply(lambda x: 0 if (type(x) != str or 'steal' not in x) else 1)

In [ ]:
cursor.execute('BEGIN;')
cursor.close()
connection.close()

In [ ]:
query = """
    SELECT description
    FROM baseball
    WHERE description LIKE '%steal%'
    GROUP BY gameid, description
"""
steal_df = pd_sql.read_sql(query, connection)

In [ ]:
steal_df.values

In [41]:
desc_query = """
    SELECT *
    FROM baseball
    WHERE rob1_outcomeid LIKE '%B%' OR rob2_outcomeid LIKE '%B%' OR rob3_outcomeid LIKE '%B%'
"""
desc_df = pd_sql.read_sql(desc_query, connection)

In [47]:
x = desc_df[desc_df['description'] == 'Marcus Semien called out on strikes.'].T
x.iloc[30:90]

,95
awayfinalrunsforinning,0
inningnumber,3
inninghalf,BOT
inningeventtype,AT_BAT
inninghalfeventsequencenumber,3
description,Marcus Semien called out on strikes.
atbateventtype,PITCH
atbateventsequencenumber,2
createdat,2016-09-07 20:27:11
updatedat,2016-09-07 20:27:15


In [29]:
for i in desc_df.values:
    if i[0].find('steal') > 0:
        print(i)

['Scooter Gennett strikes out swinging. Jonathan Villar caught stealing third.']
['Orlando Arcia strikes out swinging. Jonathan Villar steals second.']
['Jonathan Villar strikes out swinging. Jake Elmore caught stealing second.']
['Ryan Braun strikes out swinging. Jonathan Villar caught stealing second.']
['Scooter Gennett called out on strikes. Jonathan Villar steals second.']


In [ ]:
str.find()

In [ ]:
len(desc_df['description'].unique())